## ILAS - Data Mining (summer 2019) - Assignment 4
#### by Andreas Hene, Niklas Mertens, Richard Palme

In [1]:
%matplotlib inline
%load_ext line_profiler

import time
import math

import sympy
import pandas
import numpy as np
import matplotlib.pyplot as plt

First we write a function `get_repr` that returns the representative of an element `val` in the dyadic interval of level `lvl`. The representative is always chosen to be the smallest element of the dyadic array.
$$ \text{val} = j \cdot 2^{h-\text{lvl}} + \text{rest} $$
The representative can be computed by
$$ j \cdot 2^{h-\text{lvl}} $$

In [2]:
def get_repr(val, lvl, h):
    return math.floor(val / 2**(h-lvl)) * 2**(h-lvl)

Next we write a function `get_children` that returns the children dyadic intervals of a representative `u`. The intervals are again given by their representative. The children of `u` are `u` and
$$ u + 2^{h-(\text{lvl} + 1)} $$

In [3]:
def get_children(u, lvl, h, n):
    children = [u]
    if lvl != h:
        right_child = u + 2**(h - (lvl+1))
        if right_child <= n:
            children.append(right_child)
    return children

In [4]:
def base_p(u, p, size):
    res = np.zeros(size, dtype=np.int16)
    i = 0
    while u != 0:
        u, r = np.divmod(u, p)
        res[i] = r
        i += 1
    return res

def _hashfunc(u, a, b, p, k):
    return (np.dot(a, base_p(u, p, k)) + b) % p

In [5]:
for u in range(9):
    print(_hashfunc(u, [1, 2], 1, p=3, k=2), end=' ')

1 2 0 0 1 2 2 0 1 

In [14]:
def count_min_sketch(filepath, eps, delta):
    with open(filepath) as f:
        n = int(f.readline())
        m = int(f.readline())         # unused
        threshold = int(f.readline())
    
    d = math.ceil(math.log(1.0 / delta, 2))
    w = math.ceil(2.0 / eps)
    
    # find a prime p with p >= w
    p = w
    while not sympy.isprime(p):
        p += 1

    # choose k such that p^k - 1 >= n
    # i.e. k >= log(n+1, p)
    k = math.ceil(math.log(n+1, p))

    hashfuncs = []
    for _ in range(d):
        a = np.asarray([np.random.randint(p) for _ in range(k)])
        b = np.random.randint(p)
        hashfuncs.append(lambda u, a=a, b=b: _hashfunc(u, a, b, p, k))

    #hashfuncs = np.zeros((d, n+1), dtype=np.int16)
    #for i in range(d):
    #    a = np.asarray([np.random.randint(p) for _ in range(k)])
    #    b = np.random.randint(p)
    #    for j in range(n+1):
    #        hashfuncs[(i, j)] = hashfunc(j, a, b, p, k)

    hashpattern = np.zeros((n+1, d, p), dtype=np.int8)
    for u in range(n + 1):
        for i, hashfunc in enumerate(hashfuncs):
            hashpattern[(u, i, hashfunc(u))] = 1

    h = math.ceil(math.log(n, 2))
       
    # note that, since w isn't necessarily a prime number,
    # we might have to make w larger (i.e. take p instead of w)
    C = np.zeros((h+1, d, p), dtype=np.int32)

    # dtype is int16, so 2 bytes. chunksize is 10**5,
    # so one chunk is exactly 0.2 MB and
    # should fit into cache.
    chunks = pandas.read_csv(
        filepath,
        header=None,
        skiprows=3,
        squeeze=True,
        dtype=np.int16,
        delim_whitespace=True,
        chunksize=10**5
    )

    # process the stream. Each chunk is of type pandas.series
    for chunk in chunks:
        for x in chunk:
            for lvl in range(h+1):
                u = get_repr(x, lvl, h)
                #for i, hashfunc in enumerate(hashfuncs):
                    #C[(lvl, i, hashfunc(u))] += 1
                #for i in range(d):
                #    C[(lvl, i, hashfuncs[(i, u)])] += 1
                np.add(C[lvl], hashpattern[u], out=C[lvl])
    
    # now we do BFS. the values in explore_current are
    # the representatives of the dyadic arrays that
    # currently get explored on this level.
    explore_current = [0]
    explore_next = []
    approx_freq = np.zeros(d, dtype=np.int32)
    for lvl in range(h + 1):
        for u in explore_current:
            for i, hashfunc in enumerate(hashfuncs):
                approx_freq[i] = C[(lvl, i, hashfunc(u))]
            #for i in range(d):
            #    approx_freq[i] = C[(lvl, i, hashfuncs[(i, u)])]
            if approx_freq.min() >= threshold:
                explore_next.extend(get_children(u, lvl, h, n))
        explore_current = explore_next
        explore_next = []
        
    # prepare the output:
    approx_frequency = []
    for u in explore_current:
        for i, hashfunc in enumerate(hashfuncs):
            approx_freq[i] = C[(h, i, hashfunc(u))]
        #for i in range(d):
        #    approx_freq[i] = C[(h, i, hashfuncs[(i, u)])]
        approx_frequency.append(approx_freq.min())
        
    return explore_current, approx_frequency

In [16]:
filename = [
    'data/easy.txt',
    'data/large_15k.txt',
    'data/large_25k.txt',
    'data/larger_40k.txt',
    'data/largest_40k.txt',
    'data/wide_1k.txt',
]

filepath = filename[1]
#filepath = 'data/supereasy'
    
start = time.time()
result, freq = count_min_sketch(filepath, eps=0.1, delta=0.1)
end = time.time()
elapsed_time = int(round(end - start))

print(result, freq)
print('time in seconds:', elapsed_time)

[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111] [82157, 82163, 81738, 81295, 81813, 68443, 68603, 67100, 65850, 75713, 74988, 77306

In [8]:
#filepath = filename[0]
#%lprun -f count_min_sketch count_min_sketch(filepath, eps=0.1, delta=0.1)

In [9]:
def brute(filepath):
    with open(filepath) as f:
        n = int(f.readline())
        m = int(f.readline())
        t = int(f.readline())
    
    chunks = pandas.read_csv(
        filepath,
        header=None,
        skiprows=3,
        squeeze=True,
        dtype=np.int16,
        delim_whitespace=True,
        chunksize=8*10**5
    )
    freq = np.zeros(n+1, dtype=np.int32)
    for chunk in chunks:
        for x in chunk:
            freq[x] += 1         
    return [(i, f) for (i, f) in zip(range(n+1), list(freq)) if f >= t]

In [10]:
#filepath = filename[0]
    
start = time.time()
result = brute(filepath)
end = time.time()
elapsed_time = int(round(end - start))

print(result)
print('time in seconds:', elapsed_time)


[(2, 4)]
time in seconds: 0
